In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
import re
import joblib
import string

In [50]:
fake= pd.read_csv(r'C:\Users\25ikb\Downloads\FakeNewsDetection-20250302T063441Z-001\FakeNewsDetection\Fake.csv')
true = pd.read_csv(r'C:\Users\25ikb\Downloads\FakeNewsDetection-20250302T063441Z-001\FakeNewsDetection\True.csv')



In [51]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [52]:
fake['class']=0
true['class']=1

In [53]:
data = pd.concat([fake,true],axis=0)

In [54]:
data.sample(10)

,title,text,subject,date,class
19951,HUFFINGTON POST WAVES WHITE FLAG…Calls Race Fo...,This is not just a win for the Alt-Right or ...,left-news,"Sep 19, 2016",0
1766,Trump Asks Advice From The Swamp He Promised ...,"Not too long after taking office, Trump decide...",News,"April 16, 2017",0
3469,Trump LITERALLY Picks A Friend Of Vladimir Pu...,Donald Trump just did Russia a big favor to th...,News,"December 10, 2016",0
690,"Trump praises 'productive' China talks, but to...",BEIJING (Reuters) - U.S. President Donald Trum...,politicsNews,"November 10, 2017",1
10209,FACTORY WORKER RIPS INTO Speaker Paul Ryan on ...,We love this guy! Factory worker Keith Ketzler...,politics,"Aug 7, 2017",0
19658,“STOP BLAMING WHITE PEOPLE For Trump’s Win Las...,Stop blaming white people for Trumps win last ...,left-news,"Nov 9, 2016",0
8062,Obama to veto September 11 lawsuit bill by Fri...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"September 22, 2016",1
11707,BREAKING: WINNING! SENATE CONFIRMS Senator Jef...,"WITH A VOTE OF 52-47, SENATOR JEFF SESSIONS WA...",politics,"Feb 8, 2017",0
8680,Scholastic Yanks Children’s Book For Its Port...,"On Sunday, publishing company Scholastic annou...",News,"January 18, 2016",0
8646,GOP Strategist: Trump Supporters Are All ‘Cra...,When one searches for the perfect way to descr...,News,"January 20, 2016",0


In [55]:
data= data.drop(["title","subject","date"],axis=1)

In [57]:
data.reset_index(inplace=True)

In [58]:
data.drop(['index'],axis=1,inplace=True)

In [60]:
def clean_text(text):
    text = text.lower()
    text=re.sub('\[.*?\]',"",text)
    text = re.sub('\\W','' ,text)
    text= re.sub("https?:://\S+","",text)
    text = re.sub("[%s]"%re.escape(string.punctuation),"",text)
    text = re.sub("\n",'',text)
    text = re.sub("\w\d\w",'',text)
    return text



In [61]:
data["text"] = data["text"].apply(clean_text)

In [62]:
data.sample(2)

,text,class
15020,ourvisionoftheworldwherearethecatholicswhoreag...,0
37515,dublinreutersirelandarmedwithaneuvetoandinsist...,1


In [69]:
x=data['text']
y=data['class']

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)


In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)  
xv_test = vectorizer.transform(xtest)  


In [71]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced',random_state=42)
lr.fit(xv_train, ytrain)  


LogisticRegression(class_weight='balanced', random_state=42)

In [72]:
prediction= lr.predict(xv_test)
lr.score(xv_test,ytest)

0.6605790645879732

In [74]:
from sklearn.metrics import classification_report

print(classification_report(ytest, prediction, labels=[0, 1]))



              precision    recall  f1-score   support

           0       1.00      0.36      0.53      4733
           1       0.58      1.00      0.74      4247

    accuracy                           0.66      8980
   macro avg       0.79      0.68      0.63      8980
weighted avg       0.80      0.66      0.62      8980



In [68]:

joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']